### --- Day 23: LAN Party ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/23

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2024, 23);
WriteLines(inputLines);

Loading puzzle file: Day23.txt
Total lines: 3380
Max line length: 5

kf-gq
cc-pf
oc-ev
eo-ju
sq-xg


In [4]:
string[] testInputLines = [
    "kh-tc",
    "qp-kh",
    "de-cg",
    "ka-co",
    "yn-aq",
    "qp-ub",
    "cg-tb",
    "vc-aq",
    "tb-ka",
    "wh-tc",
    "yn-cg",
    "kh-ub",
    "ta-co",
    "de-co",
    "tc-td",
    "tb-wq",
    "wh-td",
    "ta-ka",
    "td-qp",
    "aq-cg",
    "wq-ub",
    "ub-vc",
    "de-ta",
    "wq-aq",
    "wq-vc",
    "wh-yn",
    "ka-de",
    "kh-ta",
    "co-tc",
    "wh-qp",
    "tb-vc",
    "td-yn",
];

In this graph-based problem we should be able to model the graph as a dictionary of node -> neighbours. To be a connected triplet, we know that `a -> b`, `b -> c`, and `c -> a`. Because its undirected, this is sufficient to establish interconnection between these three nodes.

One super-simple way to do this might be to just try all combinations?! Once we have the neighbour dictionary, testing all `[a,b,c]` combinations should be reasonably fast for a network of this size. Let's see...

In [5]:
using NeighbourLookup = SCG.Dictionary<string, SCG.HashSet<string>>;

NeighbourLookup ParseNeighbours(string[] inputLines)
{
    NeighbourLookup result = new();
    
    foreach (var line in inputLines)
    {
        var (a, b) = line.Split('-');

        result.TryAdd(a, new());
        result[a].Add(b);
        result.TryAdd(b, new());
        result[b].Add(a);
    }

    return result;
}

In [6]:
using Triplet = (string a, string b, string c);

IEnumerable<Triplet> FindMatchingTriplets(string[] inputLines)
{
    var neighbours = ParseNeighbours(inputLines);
    var nodes = neighbours.Keys.ToList();
    nodes.Sort();
    var ts = nodes.Select(n => n[0] is 't').ToArray();

    for (var i = 0; i < nodes.Count; i++)
    for (var j = i + 1; j < nodes.Count; j++)
    {
        var a = nodes[i];
        var b = nodes[j];

        if (neighbours[a].Contains(b))
        {
            for (var k = j + 1; k < nodes.Count; k++)
            {
                var c = nodes[k];
                
                if (neighbours[b].Contains(c)
                && neighbours[c].Contains(a))
                {
                    if (ts[i] || ts[j] || ts[k])
                    {
                        yield return (a, b, c);
                    }
                }
            }
        }
    }
}

In [7]:
// ... That narrows the list down to 7 sets of three inter-connected computers:

// co,de,ta
// co,ka,ta
// de,ka,ta
// qp,td,wh
// tb,vc,wq
// tc,td,wh
// td,wh,yn

foreach (var triplet in FindMatchingTriplets(testInputLines))
{
    Console.WriteLine(triplet);
}

(co, de, ta)
(co, ka, ta)
(de, ka, ta)
(qp, td, wh)
(tb, vc, wq)
(tc, td, wh)
(td, wh, yn)


In [8]:
int CountInterconnected(string[] inputLines) => FindMatchingTriplets(inputLines).Count();

var testAnswer = CountInterconnected(testInputLines);
Console.WriteLine(testAnswer);

7


In [9]:
// Find all the sets of three inter-connected computers. How many contain at
// least one computer with a name that starts with t?

var part1Answer = CountInterconnected(inputLines);
Console.WriteLine(part1Answer);

1083


In [10]:
// 1083 is correct
Ensure(1083, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/23

Ok, so for part 2, we need to find the largest set of nodes that are strongly connected to each other.

For a group of `n` nodes to be strongly connected, there needs to be `n-1` connections between them. Let's take a quick look at our theoretical maximum size, to get an idea for which approach to take.

In [12]:
void CheckProblemSize(string[] inputLines)
{
    var neighbours = ParseNeighbours(inputLines);
    var totalNodes = neighbours.Count;
    var maxNeighbours = neighbours.Values.Max(v => v.Count);

    Console.WriteLine($"Total nodes: {totalNodes}");
    Console.WriteLine($"Theoretical max connected size: {maxNeighbours + 1}");
}
CheckProblemSize(inputLines);

Total nodes: 520
Theoretical max connected size: 14


Ok, testing all combinations of 14 nodes out of 520 is too computationally expensive, so unsurprisingly our Part 1 stragegy won't work here. I think we'll need to figure out an efficient way to traverse the nodes instead.

We know that for nodes to be strongly connected, there must exist direct edges between every node. Therefore, trivially, a pair of nodes are strongly connected to each other.

For a given set of `n` strongly connected nodes, we know that adding another node to this set requires that all members of this set have an edge to the new node. Since our graph structure gives us the neighbours for each node, if we compute the set intersection of these neighbours, any nodes in this intersection can be used to grow our strongly connected set by another node. Note we can't just add all nodes from the intersection in one go: we haven't verified these nodes are connected to each other. But for `m` nodes in the intersection, we can create `m` new groups of size `n + 1`.

Given this, we can start with all pairs of connected nodes, and incrementally grow these sets until we find the largest one. Hopefully that's computationally feasible, let's see!

In [13]:
string FindLargestConnectedSet(string[] inputLines)
{
    var allNeighbours = ParseNeighbours(inputLines);

    Queue<string[]> setQueue = new();
    foreach (var (node, neighbour) in allNeighbours)
    {
        // Keep all the nodes in alphabetical order: that way we don't consider
        // [a,b,c] and [b,c,a], which are equivalent
        foreach (var n in neighbour.Where(n => n.CompareTo(node) > 0))
        {
            setQueue.Enqueue([node, n]);
        }
    }

    string[] maxSet = [];
    while (setQueue.TryDequeue(out var set))
    {
        if (set.Length > maxSet.Length)
        {
            Console.WriteLine($"Found a new max length set of {set.Length}");
            maxSet = set;
        }

        HashSet<string> nextNodes = new(allNeighbours[set[0]]);
        foreach (var member in set[1..]) {
            nextNodes.IntersectWith(allNeighbours[member]);
        }

        var lastNode = set[^1];
        foreach (var nextNode in nextNodes.Where(nn => nn.CompareTo(lastNode) > 0))
        {
            setQueue.Enqueue([..set, nextNode]);
        }
    }

    return string.Join(",", maxSet);
}

In [14]:
// In this example, the password would be co,de,ka,ta.

var part2TestAnswer = FindLargestConnectedSet(testInputLines);
Console.WriteLine(part2TestAnswer);

Found a new max length set of 2
Found a new max length set of 3
Found a new max length set of 4
co,de,ka,ta


In [15]:
// What is the password to get into the LAN party?

var part2Answer = FindLargestConnectedSet(inputLines);
Console.WriteLine(part2Answer);

Found a new max length set of 2
Found a new max length set of 3
Found a new max length set of 4
Found a new max length set of 5
Found a new max length set of 6
Found a new max length set of 7
Found a new max length set of 8
Found a new max length set of 9
Found a new max length set of 10
Found a new max length set of 11
Found a new max length set of 12
Found a new max length set of 13
as,bu,cp,dj,ez,fd,hu,it,kj,nx,pp,xh,yu


In [16]:
// as,bu,cp,dj,ez,fd,hu,it,kj,nx,pp,xh,yu is correct!
Ensure("as,bu,cp,dj,ez,fd,hu,it,kj,nx,pp,xh,yu", part2Answer);